# 第三章 字典和集合

字典(dict)和集合(set)的实现都是基于散列表

## 3.1 泛映射类型

## 3.2 字典推导

字典推导（dictcomp）可以从任何以键值对作为元素的可迭代对象中构建出字典。

In [1]:
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'),
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan')
]
country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

In [2]:
{code: country.upper() for country, code in country_code.items() if code < 66}


{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}

## 3.3 常见的映射方法

映射类型的方法其实很丰富: `dict`、 `defaultdict` 和 `OrderedDict`

后面两个数据类型是 `dict` 的变种，位于 `collections` 模块内

| method |dict| defaultdict | OrderedDict| help|
| :----: | :----: | :----: | :----: | :----: |
 |`d.clear()` |•| • |• | 移除所有元素| |`d.__contains__(k)`| •| • |•| 检查 k 是否在 d 中| |`d.copy()`| • | •| • | 浅复制|
| `d.__copy__()` | - | • | - |用于支持 copy.copy|
|`d.default_factory`|- | •| - | 在 `__missing__` 函数中被调用的函数，用以给未找到的元素设置值*|
| `d.__delitem__(k)`| •| •| •| `del d[k]`，移除键为 `k` 的元素|
|`d.fromkeys(it,[initial])`| •| •| •|将迭代器 it 里的元素设置为映射里的键，如果有 initial 参数，就把它作为这些键对应的值（默认是 None）|
|`d.get(k,[default])`| •| •| •| 返回键 k 对应的值，如果字典里没有键 k，则返回 None 或者 default|
|`d.__getitem__(k)`| •| •| •| 让字典 d 能用 `d[k]` 的形式返回键 k 对应的值|
|`d.items()`| •| •| •| 返回 d 里所有的键值对|
|`d.__iter__()`| •| •| •| 获取键的迭代器|
|`d.keys()`| •| •| •| 获取所有的键|
|`d.__len__()`| •| •| •| 可以用 `len(d)`的形式得到字典里键值对的数量|
|`d.__missing__(k)`| - | •| -|  当 `__getitem__` 找不到对应键的时候，这个方法会被调用|
|`d.move_to_end(k,[last])`| -| - | •| 把键为 k 的元素移动到最靠前或者最靠后的位置（last 的默认值是 True）
|`d.pop(k, [defaul])`|•| •| •| 返回键 k 所对应的值，然后移除这个键值对。如果没有这个键，返回 None 或者 defaul|
|`d.popitem()`| •| •| •| 随机返回一个键值对并从字典里移除它#|
|`d.__reversed__()`|-|-| •| 返回倒序的键的迭代器|
|`d.setdefault(k,[default])`| •| •| •| 若字典里有键k，则把它对应的值设置为 default，然后返回这个值；若无，则让 `d[k] =default`，然后返回 default|
|`d.__setitem__(k,v)`| •| •| •| 实现 `d[k] = v` 操作，把 k 对应的值设为v|
|`d.update(m,[**kargs])`| •| •| •| m 可以是映射或者键值对迭代器，用来更新 d 里对应的条目|
|`d.values()`| •| •| •| 返回字典里的所有值|

上面的表格中， `update` 方法处理参数 m 的方式，是典型的“鸭子类型”。
函数首先检查 m 是否有 keys 方法，如果有，那么 update 函数就把它当作映射对象来处理。
否则，函数会退一步，转而把 m 当作包含了键值对 (key, value) 元素的迭代器。
Python 里大多数映射类型的构造方法都采用了类似的逻辑，因此你既可以用一个映射对象来新建一个映射对象，也可以用包含 (key, value) 元素的可迭代对象来初始化一个映射对象

当查找字典某个值不存在的时候，会报错，如果不想抛出异常，可以给一个default的值

`d.get(k,default)` 或 `d.setdefault(k,default)`

后者会往d中插入一个键值对`(k:default)`

## 3.4　映射的弹性键查询

为了方便起见，就算某个键在映射里不存在，我们也希望在通过这个键读取值的时候能得到一个默认值

两个方法: 一个是通过 `defaultdict` 这个类型而不是普通的 `dict`，另一个是给自己定义一个 dict 的子类，然后在子类中实现`__missing__`方法

### 3.4.1 `defaultdict`：处理找不到的键的一个选择

实例化一个 `defaultdict` 的时候，需要给构造方法提供一个可调用对象，
这个可调用对象会在 `__getitem__` 碰到找不到的键的时候被调用，让`__getitem__` 返回某种默认值
